In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

In [ ]:
import os
#input_file = 'mariel-1.npy'
#input_file = 'carrie-10-min-scaled.npy'
input_file = 'carrie-10-mins.npy'

X0 = np.load(os.path.join("data/npy", input_file))
#X0 = np.load(os.path.join(input_file))

if input_file == 'mariel-1.npy':
    X0 = X0.transpose((1,0,2))
    X0 = X0[1300:7000]
    X0[:,:,2] *= -1
    X = X0.copy()
    X -= X.mean(axis=(0,1))
    X *= 0.5
elif input_file == 'carrie-10-min-scaled.npy':
    X0 = X0.transpose((1,0,2))
    X = X0.copy()
    X[:,:,1] = X0[:,:,2]
    X[:,:,2] = X0[:,:,1]
    X = X[100:]
    X[:,:,:2] -= 0.25
if input_file == 'carrie-10-mins.npy':
    X0 = X0.transpose((1,0,2))
    X0 = X0[100:]
    X0[:,:,2] *= -1
    X = X0.copy()
    X -= X.mean(axis=(0,1))
    X *= 0.5
    

# set to True if you want to cluster the 55 vertices into a
# reduced set of 8 vertices
do_cluster = False
if do_cluster:
    from sklearn.cluster import KMeans
    
    nc = 8
    km = KMeans(n_clusters=nc).fit(X.transpose((1,2,0)).reshape(X.shape[1],-1))
    Xcl = np.zeros((X.shape[0], nc, 3))
    for i in range(nc):
        Xcl[:,i] = X[:,np.argwhere(km.labels_==i)[:,0]].mean(axis=1)
    X = Xcl
    
print(X.shape)

In [ ]:
plt.plot(X.mean(axis=1));
plt.figure()
plt.hist([X[:,:,0].flatten(),X[:,:,1].flatten(),X[:,:,2].flatten()], histtype='step');

In [ ]:
print(X.max(axis=(0,1)))
print(X.min(axis=(0,1)))

In [ ]:
print(X.mean(axis=(0,1)))
print(X.std(axis=(0,1)))

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d.axes3d as p3
from mpl_toolkits.mplot3d.art3d import juggle_axes
from matplotlib import animation
from IPython.display import HTML

# ask matplotlib to plot up to 2^128 frames in animations
matplotlib.rcParams['animation.embed_limit'] = 2**128

def update_points(time, points, df):
  '''
  Callback function called by plotting function below. Mutates the vertex
  positions of each value in `points` so the animation moves
  @param int time: the index of the time slice to visualize within `df`
  @param mpl_toolkits.mplot3d.art3d.Path3DCollection points: the actual
    geometry collection whose internal values this function mutates to move
    the displayed points
  @param numpy.ndarray df: a numpy array with the following three axes:
    df.shape[0] = n_vertices
    df.shape[1] = n_time_slices
    df.shape[2] = n_dimensions
  '''
  points._offsets3d = juggle_axes(df[time,:,0], df[time,:,1], df[time,:,2], 'z')

def animate(seq, frames=None, axis_min=-0.75, axis_max=0.75, speed=45):
    if frames is None:
        frames = len(seq)
    fig = plt.figure()
    ax = p3.Axes3D(fig)
    ax.set_xlim(axis_min, axis_max)
    ax.set_ylim(axis_min, axis_max)
    ax.set_zlim(axis_min, axis_max)
    points = ax.scatter(seq[0,:,0], seq[0,:,1], seq[0,:,2], depthshade=False)
    return animation.FuncAnimation(
        fig,
        update_points,
        frames,
        interval=speed,
        fargs=(points, seq),
        blit=False,
    ).to_jshtml()

def plot_pose(x, ax=None):
    if ax is None:
        fig = plt.figure()
        ax = p3.Axes3D(fig)
    ax.set_xlim(-0.75,0.75)
    ax.set_ylim(-0.75,0.75)
    ax.set_zlim(-0.75,0.75)
    ax.scatter(x[:,0], x[:,1], x[:,2])
    return ax

def plot_stick(x, edges, ax=None, figsize=None):
    if ax is None:
        fig = plt.figure(figsize=figsize)
        ax = p3.Axes3D(fig)
    ax.set_xlim(-0.75,0.75)
    ax.set_ylim(-0.75,0.75)
    ax.set_zlim(-0.75,0.75)
    #ax.scatter(x[:,0], x[:,1], x[:,2])
    for e in edges:
        ax.plot(np.linspace(x[e[0],0],x[e[1],0],10),np.linspace(x[e[0],1],x[e[1],1],10),np.linspace(x[e[0],2],x[e[1],2],10))
    return ax

In [ ]:
vdist_mean = np.zeros((X.shape[1],X.shape[1]))
vdist_var = np.zeros_like(vdist_mean)
for i in range(X.shape[1]):
    for j in range(i+1,X.shape[1]):
        vdist = np.sum((X[:,i]-X[:,j])**2, axis=-1)
        vdist_mean[i,j] = vdist_mean[j,i] = vdist.mean()
        vdist_var[i,j] = vdist_var[j,i] = vdist.var(ddof=1)
vdist_std = np.sqrt(vdist_var)

vdist_mean_q5, vdist_mean_q95 = np.quantile(vdist_mean[vdist_mean>0], (0.05,0.95))
vdist_var_q5, vdist_var_q95 = np.quantile(vdist_var[vdist_var>0], (0.05,0.95))
vdist_std_q5, vdist_std_q95 = np.quantile(vdist_std[vdist_std>0], (0.05,0.95))
print("pairwise mean quantiles:  5th=%.2e  95th=%.2e" % (vdist_mean_q5, vdist_mean_q95))
print("pairwise std. quantiles:  5th=%.2e  95th=%.2e" % (vdist_std_q5, vdist_std_q95))
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.hist(np.log10(vdist_mean[vdist_mean>0]), bins=40);
plt.axvline(np.log10(vdist_mean_q5), color='red', lw=1);
plt.axvline(np.log10(vdist_mean_q95), color='red', lw=1);
plt.xlabel("log10(pairwise mean)", fontsize=14)
plt.subplot(1,2,2)
plt.hist(np.log10(vdist_std[vdist_std>0]), bins=40);
plt.xlabel("log10(pairwise std.)", fontsize=14);
plt.axvline(np.log10(vdist_std_q5), color='red', lw=1);
plt.axvline(np.log10(vdist_std_q95), color='red', lw=1);

In [ ]:
plt.figure(figsize=(12,4.5))
plt.subplot(1,2,1)
plt.imshow(vdist_mean, vmin=vdist_mean_q5, vmax=vdist_mean_q95, norm=LogNorm())
plt.colorbar()
plt.title("pairwise mean", fontsize=14)
plt.subplot(1,2,2)
plt.imshow(vdist_std, vmin=vdist_std_q5, vmax=vdist_mean_q95, norm=LogNorm())
plt.title("pairwise std.", fontsize=14);
plt.colorbar();

In [ ]:
upper_triangle = np.triu_indices_from(vdist_var, k=1)
variances = vdist_var[upper_triangle]
vtx_pairs = sorted(zip(*upper_triangle), key=lambda p: vdist_var[p[0],p[1]])

In [ ]:
n_rigid = 140
#n_rigid = 7
plt.figure(figsize=(8,3))
plt.plot(sorted(vdist_var[upper_triangle]))
plt.yscale('log')
plt.ylabel('pairwise variance', fontsize=14)
plt.xlabel('pair #', fontsize=14)
plt.axvline(n_rigid, color='red', lw=1);
frame_num = 500
ax = plot_stick(X[frame_num], vtx_pairs[:n_rigid], figsize=(8,8))
plot_pose(X[frame_num], ax);
plt.title("Reference pose (frame %d)"%frame_num, fontsize=16)
ax.get_figure

In [ ]:
import os
import tensorflow as tf
# config for Gatsby cores
if 'gatsby' in os.environ['HOSTNAME']:

    # specify target gpu device
    os.environ['CUDA_VISIBLE_DEVICES'] = '0' # either '0' or '1' to utilize Titan X GPUs

    # allow dynamic GPU allocation
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    session = tf.Session(config=config)
    
import keras.backend as K
from keras import layers
from keras.models import Model
from keras.utils import to_categorical
from keras import objectives

In [ ]:
def mk_model(lookback=60, npred=4, n_cells=(48,48,48), n_cells_disc=(48,48,48,), noise_dim=128, center_seq=False, n_rigid=0, use_mse=False):
    K.clear_session()
    probe = None
    
    input_seq = layers.Input((lookback, X.shape[1], X.shape[2]))
    H = input_seq
    
    input_noise = layers.Input((noise_dim,))
    Hn = input_noise
    
    
    # first, translate each input sequences in the batch so it's centered at 0,0
    if center_seq:
        # mask [1,1,0] to ignore z-axis shift
        offsets = layers.Lambda(lambda x: K.constant([[[[1,1,0]]]])*K.mean(x,axis=(1,2),keepdims=True))(H)
        H = layers.Subtract()([H,offsets])
        probe = Model(input_seq, H)
        
    H = layers.Reshape((lookback, X.shape[1]*X.shape[2]))(H)
    
    for nc in n_cells:
        H = layers.CuDNNLSTM(nc, return_sequences=True)(H)
    #H = layers.CuDNNLSTM(n_cells[-1])(H)
    H = layers.Cropping1D((lookback-npred,0))(H)
    
    Hn = layers.Dense(npred*n_cells[-1])(Hn)
    Hn = layers.LeakyReLU()(Hn)
    Hn = layers.Reshape((npred,n_cells[-1]))(Hn)
    
    H = layers.Add()([H,Hn])
    
    H = layers.Dense(X.shape[1]*X.shape[2])(H)
    
    H = layers.Reshape((npred,X.shape[1],X.shape[2]))(H)
    
    # if we shifted before processing, shift back before the output
    if center_seq:
        H = layers.Add()([H, offsets])
    
    output_seq = H
    
    generator = Model([input_seq, input_noise], output_seq)
    
    disc_input1 = H1 = layers.Input((lookback, X.shape[1], X.shape[2]))
    disc_input2 = H2 = layers.Input((npred, X.shape[1], X.shape[2]))
    
    if center_seq:
        # mask [1,1,0] to ignore z-axis shift
        doffsets = layers.Lambda(lambda x: K.constant([[[[1,1,0]]]])*K.mean(x,axis=(1,2),keepdims=True))(disc_input1)
        H1 = layers.Subtract()([H1,doffsets])
        H2 = layers.Subtract()([H2,doffsets])
    
    H1 = layers.Reshape((lookback, X.shape[1]*X.shape[2]))(H1)
    H2 = layers.Reshape((npred, X.shape[1]*X.shape[2]))(H2)
    
    for nc in n_cells_disc[:-1]:
        L = layers.CuDNNLSTM(nc, return_sequences=True)
        H1 = L(H1)
        H2 = L(H2)
    
    H = layers.Concatenate(axis=1)([H1,H2])
    H = layers.CuDNNLSTM(n_cells_disc[-1])(H)
    
    H2 = layers.CuDNNLSTM(n_cells_disc[-1])(H2)
    
    H = layers.Concatenate()([H,H2])
        
    def s(l):
        return layers.LeakyReLU()(l)
    H = layers.Dense(128)(H)
    H = s(H)
    H = layers.Dense(1, activation='sigmoid', use_bias=False)(H)
    disc_output = H
    
    discriminator = Model([disc_input1,disc_input2], disc_output)
    discriminator.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
    
    discriminator.trainable = False
    input_true = layers.Input((npred, X.shape[1], X.shape[2]))
    
    fg_output = generator([input_seq, input_noise])
    full_output = discriminator([input_seq,fg_output])
    full = Model([input_seq, input_noise], full_output)
    
    full.hp_rigidity_weight = K.variable(1.0)
    full.hp_xe_weight = K.variable(1.0)
    full.hp_mse_weight = K.variable(1.0)    

    
    def full_loss(y_true, y_pred):
        y_ones = K.ones((K.shape(full_output)[0],1))
        xe_loss = K.mean(K.binary_crossentropy(y_ones,full_output,), axis=-1)
        
        if use_mse:
            mse_loss = K.mean(K.square(y_true - fg_output),axis=(-3,-2,-1))
        
        rigidity_loss = 0
        for i,j in vtx_pairs[:n_rigid]:
            a = fg_output[:,:,i]
            b = fg_output[:,:,j]
            d = vdist_mean[i,j]
            v = vdist_var[i,j]
            D = K.sum(K.square(a-b),axis=-1)
            rigidity_loss += K.square(D-d)/v    
        if n_rigid:
            rigidity_loss /= n_rigid
            rigidity_loss = K.mean(rigidity_loss, axis=-1)
                
        loss_total = full.hp_xe_weight*xe_loss
        if n_rigid:
            loss_total += full.hp_rigidity_weight*rigidity_loss
        if use_mse:
            loss_total += full.hp_mse_weight*mse_loss
        return loss_total
    
    full.compile(loss=full_loss, optimizer='adam')
    discriminator.trainable = True
    
    return generator, discriminator, full, probe

In [ ]:
lookback = 24
npred = 1
noise_dim = 32
center_seq = True
n_cells = (96,64,32)
n_cells_disc = (48,48,48,)
use_mse = False

gen, disc, full, probe = mk_model(lookback=lookback, npred=npred,
                                  n_cells=n_cells, n_cells_disc=n_cells_disc,
                                  noise_dim=noise_dim, center_seq=center_seq,
                                  n_rigid=n_rigid, use_mse=use_mse)
gen.summary()
disc.summary()
disc.trainable = False
full.summary()
disc.trainable = True

d_losses = []
g_losses = []
rscores = dict()

In [ ]:
K.set_value(full.optimizer.lr, 1e-4)
K.set_value(disc.optimizer.lr, 1e-5)

K.set_value(full.hp_rigidity_weight, 1e-4)
K.set_value(full.hp_xe_weight, 1)
K.set_value(full.hp_mse_weight, 0)

In [ ]:
epochs = 1
batch_size = 128
do_rotation = True

nbatches = (len(X)-batch_size*lookback-npred)//batch_size

#y_real = to_categorical(np.ones(batch_size),2)
#y_fake = to_categorical(np.zeros(batch_size),2)
y_real = np.ones(batch_size)
y_fake = np.zeros(batch_size)
print("Nbatches =", nbatches)
for iepoch in range(epochs):
    offsets = np.random.choice(len(X)-batch_size*lookback-npred, replace=False, size=(nbatches,batch_size))
    
    d_loss = 0
    g_loss = 0
    for ibatch in range(nbatches):
        batch_idxs = offsets[ibatch].repeat(lookback).reshape(batch_size,lookback) + np.arange(lookback)
        truth_idxs = offsets[ibatch].repeat(npred).reshape(batch_size,npred) + np.arange(npred) + lookback
        
        x1_real = X[batch_idxs]
        x2_real = X[truth_idxs]
        if do_rotation:
            theta = 2*np.pi*np.random.rand()
            c,s = np.cos(theta), np.sin(theta)
            rot = np.array([[c,-s,0],[s,c,0],[0,0,1]])
            x1_real = rot.dot(x1_real.transpose((0,1,3,2))).transpose((1,2,3,0))
            x2_real = rot.dot(x2_real.transpose((0,1,3,2))).transpose((1,2,3,0))
        
        
        noise = np.random.normal(size=(batch_size,noise_dim))
        x2_fake = gen.predict([x1_real, noise])
        
        disc.trainable = True
        l, a = disc.train_on_batch([x1_real, x2_real], y_real)
        d_loss += 0.5*l
        l,a = disc.train_on_batch([x1_real, x2_fake], y_fake)
        d_loss += 0.5*l
        
        # TODO: figure out if it makes sense to redo translations/rotations before
        # the generator training step.
        
        if do_rotation:
            theta = 2*np.pi*np.random.rand()
            c,s = np.cos(theta), np.sin(theta)
            rot = np.array([[c,-s,0],[s,c,0],[0,0,1]])
            x1_real = rot.dot(x1_real.transpose((0,1,3,2))).transpose((1,2,3,0))
            x2_real = rot.dot(x2_real.transpose((0,1,3,2))).transpose((1,2,3,0))
        
        disc.trainable = False
        noise = np.random.normal(size=(batch_size,noise_dim))
        g_loss += full.train_on_batch([x1_real, noise], x2_real)
        
    d_losses.append(d_loss/nbatches)
    g_losses.append(g_loss/nbatches)
    print("Epoch %d/%d: L(d)=%.2e L(g)=%.2e" % (iepoch, epochs, d_losses[-1], g_losses[-1]))

In [ ]:
plt.plot(d_losses[:], label='disc')
plt.plot(g_losses[:], label='gen')
plt.legend();
#plt.xlim(left=250)
#plt.ylim(top=0.75, bottom=.65)
#plt.yscale('log');

In [ ]:
#gen.load_weights('models/gan-gen-e1656.wt.h5')
#disc.load_weights('models/gan-disc-e1656.wt.h5')

In [ ]:
iprompt = np.random.randint(len(X)-batch_size*lookback-1)
ngen = 9*lookback

gen_seq = np.zeros((lookback+ngen, X.shape[1], X.shape[2]))
gen_seq[:lookback] = X[iprompt:iprompt+lookback]

for i in range(ngen):
    next_frame = gen.predict([np.expand_dims(gen_seq[i:i+lookback],0),np.random.normal(size=(1,noise_dim))])[0,0]
    gen_seq[i+lookback] = next_frame

# plot the first generated frame
#plot_pose(gen_seq[lookback])
ax = plot_stick(gen_seq[lookback], vtx_pairs[:n_rigid], figsize=(8,8))
plot_pose(gen_seq[lookback], ax)
plt.title("Epoch %d"%(len(g_losses)-1), fontsize=16)
plt.figure(figsize=(9,4))
plt.subplot(1,2,1)
plt.title('real')
plt.hist([gen_seq[:lookback,:,0].flatten(),
          gen_seq[:lookback,:,1].flatten(),
          gen_seq[:lookback,:,2].flatten()], histtype='step', bins=20, range=(-1.5,1.5));
plt.subplot(1,2,2)
plt.title('gen')
plt.hist([gen_seq[lookback:,:,0].flatten(),
          gen_seq[lookback:,:,1].flatten(),
          gen_seq[lookback:,:,2].flatten()], histtype='step', bins=20, range=(-1.5,1.5));

dm = np.zeros_like(vdist_mean)
dv = np.zeros_like(vdist_var)
for i in range(X.shape[1]):
    for j in range(i+1,X.shape[1]):
        dm[i,j] = dm[j,i] = np.sqrt((gen_seq[lookback:,i]-gen_seq[lookback:,j])**2).sum(axis=-1).mean()
        dv[i,j] = dv[j,i] = np.sqrt((gen_seq[lookback:,i]-gen_seq[lookback:,j])**2).sum(axis=-1).var(ddof=1)
plt.figure(figsize=(9,4))
plt.subplot(1,2,1)
plt.imshow(vdist_mean, vmin=0, vmax=vdist_mean.max())
plt.title('real')
plt.subplot(1,2,2)
plt.title('gen')
plt.imshow(dm, vmin=0, vmax=vdist_mean.max())
plt.figure(figsize=(9,4))
plt.subplot(1,2,1)
plt.imshow(np.sqrt(vdist_var), vmin=0, vmax=np.sqrt(vdist_var).max())
plt.title('real')
plt.subplot(1,2,2)
plt.title('gen')
plt.imshow(np.sqrt(dv), vmin=0, vmax=np.sqrt(vdist_var).max());

rigidity_score = 0
for i,j in vtx_pairs[:n_rigid]:
    rigidity_score += ((np.sum((x2_fake[:,:,i]-x2_fake[:,:,j])**2,axis=-1)-vdist_mean[i,j])**2)/vdist_var[i,j]
if n_rigid:
    rigidity_score /= n_rigid
rigidity_score = rigidity_score.mean()

print("Rigidity score: %.2e"%rigidity_score)
rscores[len(g_losses)] = rigidity_score

plt.figure();
rs = np.array(sorted(rscores.items()))
plt.plot(rs[:,0], rs[:,1])
plt.xlim(left=0)
plt.yscale('log')

In [ ]:
HTML(animate(gen_seq))